### Importar librerias

In [0]:
!pip install -q twitterscraper
!pip install -q twint

In [0]:
from twitterscraper.query import query_user_info #informacion de usuarios
import datetime as dt ## para el manejo de las fechas
from multiprocessing import Pool #procesamiento paralelo
import pandas as pd ### para el manejo de datframes
import numpy as np
import twint

In [0]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/Alertas_Negativas_Twitter/notebooks")

### Scrapear información de los usuarios - TwitterScrapper

In [0]:
def get_user_info(twitter_user):
    try:
      user_info = query_user_info(user= twitter_user)
      twitter_user_data = {}
      twitter_user_data["screen_name"] = user_info.user
      twitter_user_data["username"] = user_info.full_name
      twitter_user_data["location"] = user_info.location
      twitter_user_data["blog"] = user_info.blog
      twitter_user_data["date_joined"] = user_info.date_joined
      twitter_user_data["user_id"] = user_info.id
      twitter_user_data["num_tweets"] = user_info.tweets
      twitter_user_data["following"] = user_info.following
      twitter_user_data["followers"] = user_info.followers
      twitter_user_data["likes"] = user_info.likes
      twitter_user_data["lists"] = user_info.lists
    except:
      twitter_user_data="None"
    return twitter_user_data

In [18]:
users=pd.read_pickle("../data/raw/df_tweets_total.pkl")["screen_name"].unique().tolist()
users[0:10]

['AliGalindez',
 'miguel_infinito',
 'joseluis6121',
 'jorge8128',
 'HallaziLuis',
 'alfremartin15',
 'CossioLuigi',
 'percy_osor',
 'JoelChavezMatos',
 'AlvaFiddler']

In [0]:
twitter_user_info=[]
pool = Pool(40)    
for user in pool.map(get_user_info,users):
  if user!="None":
    twitter_user_info.append(user)
df_user_info = pd.DataFrame(twitter_user_info)
display(df_user_info)

In [0]:
### configurando data
meses={"Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,"Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12}
def replace(x):
  val=x.split("-")[1].split()[1]
  x=x.split("-")[1].replace(val,str(meses[val])).strip().replace(" ","/")
  return x
aux2=df_user_info["date_joined"].apply(lambda x : replace(x)).apply(lambda x : dt.datetime.strptime(x, '%d/%m/%Y'))
df_user_info["date_joined"]=aux2
df_user_info.sort_values(by="date_joined",ascending=False,inplace=True)
df_user_info=df_user_info.set_index("screen_name").reset_index().copy()

In [61]:
df_user_info.drop(labels="lists",axis=1,inplace=True)
df_user_info

,screen_name,username,location,blog,date_joined,user_id,num_tweets,following,followers,likes
0,FelizDiaLima,#FelizDíaLima. 👑 👑 👑 ⭐️,,,2020-01-16,1218063509549993986,91,99,5,165
1,JuanCar99215124,Juan Carlos Miranda,,,2020-01-13,1216755584080347137,22,34,3,137
2,cocorocovillano,coqui villano diarrea,Jesus maria,,2020-01-10,1215664651813388289,200,249,49,342
3,CarasasJonathan,Jonathan Carasas C.,"Montevideo, Uruguay",elojocriticojcc.blogspot.com,2019-12-27,1210664626666520581,406,78,8,228
4,pedropablofb,Pedro Pablo,,,2019-12-26,1210126217539706886,5,99,4,2
...,...,...,...,...,...,...,...,...,...,...
10681,blogdenotas,Bruno Ortiz B. (Blogdenotas),"Lima, Perú",blogdenotas.pe,2007-04-04,3450061,80122,3099,14799,58550
10682,ocram,☄️ Marco Sifuentes,"Madrid, Comunidad de Madrid",es.wikipedia.org/wiki/Marco_Sif…,2007-04-03,3369151,116346,1907,174894,879
10683,miguelus,Migue,Ciudadano del mundo,bit.ly/2WQGOI6,2007-03-30,3006001,54856,2312,2364,28183
10684,Nest0R,|Mal0,Lima,,2007-03-24,2108371,10687,213,371,338


In [0]:
df_user_info.to_pickle("../data/raw/df_user_info.pkl")

### Scrapear Informacion de los usuario -- TWINT

In [0]:
users
c = twint.Config()
c.Store_csv = True
c.Output = "../data/raw/df_user_info_twint.csv"
c.User_full=True
for i in users:
  c.Username = i
  twint.run.Lookup(c)

In [81]:
dict_={"username":"screen_name","name":"username","location":"location","url":"blog",
        "join_date":"date_joined","id":"user_id","tweets":"num_tweets","following":"following",
        "followers":"followers","likes":"likes"}
df_user_info_twint=pd.read_csv("../data/raw/df_user_info_twint.csv",sep=",")
df_user_info_twint=df_user_info_twint[list(dict_.keys())]
df_user_info_twint.rename(columns=dict_,inplace=True)
print(df_user_info_twint.shape)
df_user_info_twint.head()

(7920, 10)


,screen_name,username,location,blog,date_joined,user_id,num_tweets,following,followers,likes
0,sientetrujillo,Siente Trujillo,"Trujillo, Perú",http://sientetrujillo.com,30 Apr 2012,567366294,694,349,2437,663
1,ROApuno,Onda Azul,Puno - Perú,https://radioondaazul.com/,26 Nov 2009,92793117,13022,108,2665,500
2,Gerardo33247325,Karl Mack Von Gerard,NaN,NaN,28 Nov 2019,1200211284647497728,16,13,0,195
3,omar_gavidia,Omar Gavidia™ 🇵🇪,Lima,https://omargavidiablog.wordpress.com/,11 Apr 2010,131762398,48549,603,1457,4303
4,WalTer_tz,Walter__✍🏻,Tomorrowland - Perú,http://mediooccidente.wordpress.com/,25 Jun 2009,50742066,30265,6861,7408,2256


In [82]:
## configurando data a datetime
meses={"Jan":1,"Feb":2,"Mar":3,"Apr":4,"May":5,"Jun":6,"Jul":7,"Aug":8,"Sep":9,"Oct":10,"Nov":11,"Dec":12}
def replace(x):
  val=x.split(" ")[1]
  x=x.replace(val,str(meses[val])).strip().replace(" ","/")
  return x
aux2=df_user_info_twint["date_joined"].apply(lambda x : replace(x)).apply(lambda x : dt.datetime.strptime(x, '%d/%m/%Y'))
df_user_info_twint["date_joined"]=aux2
print(df_user_info_twint.shape)
df_user_info_twint.head()

(7920, 10)


,screen_name,username,location,blog,date_joined,user_id,num_tweets,following,followers,likes
0,sientetrujillo,Siente Trujillo,"Trujillo, Perú",http://sientetrujillo.com,2012-04-30,567366294,694,349,2437,663
1,ROApuno,Onda Azul,Puno - Perú,https://radioondaazul.com/,2009-11-26,92793117,13022,108,2665,500
2,Gerardo33247325,Karl Mack Von Gerard,NaN,NaN,2019-11-28,1200211284647497728,16,13,0,195
3,omar_gavidia,Omar Gavidia™ 🇵🇪,Lima,https://omargavidiablog.wordpress.com/,2010-04-11,131762398,48549,603,1457,4303
4,WalTer_tz,Walter__✍🏻,Tomorrowland - Perú,http://mediooccidente.wordpress.com/,2009-06-25,50742066,30265,6861,7408,2256


### UserInfo Final

In [83]:
df_user_info_final=pd.concat([df_user_info,df_user_info_twint],axis=0)
df_user_info_final.drop_duplicates(subset="screen_name",inplace=True)
df_user_info_final

,screen_name,username,location,blog,date_joined,user_id,num_tweets,following,followers,likes
0,FelizDiaLima,#FelizDíaLima. 👑 👑 👑 ⭐️,,,2020-01-16,1218063509549993986,91,99,5,165
1,JuanCar99215124,Juan Carlos Miranda,,,2020-01-13,1216755584080347137,22,34,3,137
2,cocorocovillano,coqui villano diarrea,Jesus maria,,2020-01-10,1215664651813388289,200,249,49,342
3,CarasasJonathan,Jonathan Carasas C.,"Montevideo, Uruguay",elojocriticojcc.blogspot.com,2019-12-27,1210664626666520581,406,78,8,228
4,pedropablofb,Pedro Pablo,,,2019-12-26,1210126217539706886,5,99,4,2
...,...,...,...,...,...,...,...,...,...,...
7689,liamgula21,Piero Flores,NaN,NaN,2015-09-25,3775686195,5,7,0,0
7744,jorgechavezbeng,Jorge,NaN,NaN,2016-02-20,701224330966003718,9,23,0,3
7803,eiseneirlov,Eisen Eirlov...,NaN,NaN,2017-04-23,856242905324957696,26,100,0,15
7805,jcarlosdj2014,Jorge Carlos,NaN,NaN,2014-02-13,2342386429,4,4,5,0


In [84]:
### cargar los seguidores de la cuenta
df_usernames_followers=pd.read_csv("../data/raw/df_usernames_followers.csv",sep=",").rename(columns={"username":"screen_name"})
df_usernames_followers["is_followers"]=[1 for _ in range(0,len(df_usernames_followers))]
print(df_usernames_followers.shape)
df_usernames_followers.head()

(168386, 2)


,screen_name,is_followers
0,evlinpilar,1
1,juli3033,1
2,mariangelescq,1
3,corplumbersac,1
4,irenesalazarsa3,1


In [85]:
###  cargar a quienes sigue la cuenta
df_usernames_following=pd.read_csv("../data/raw/df_usernames_following.csv",sep=",").rename(columns={"username":"screen_name"})
df_usernames_following["is_following"]=[1 for _ in range(0,len(df_usernames_following))]
print(df_usernames_following.shape)
df_usernames_following.head()

(198, 2)


,screen_name,is_following
0,fondoflar,1
1,bis_org,1
2,premioslatinum,1
3,elcomercio_peru,1
4,cemla,1


In [86]:
### unir las tres bases, df_user_info_final + df_usernames_followers + df_usernames_following
df_user_info_final=pd.merge(df_user_info_final,df_usernames_followers,how="left",on="screen_name")
df_user_info_final=pd.merge(df_user_info_final,df_usernames_following,how="left",on="screen_name")
print(df_user_info_final.shape)
df_user_info_final.head()

(10852, 12)


,screen_name,username,location,blog,date_joined,user_id,num_tweets,following,followers,likes,is_followers,is_following
0,FelizDiaLima,#FelizDíaLima. 👑 👑 👑 ⭐️,,,2020-01-16,1218063509549993986,91,99,5,165,NaN,NaN
1,JuanCar99215124,Juan Carlos Miranda,,,2020-01-13,1216755584080347137,22,34,3,137,NaN,NaN
2,cocorocovillano,coqui villano diarrea,Jesus maria,,2020-01-10,1215664651813388289,200,249,49,342,NaN,NaN
3,CarasasJonathan,Jonathan Carasas C.,"Montevideo, Uruguay",elojocriticojcc.blogspot.com,2019-12-27,1210664626666520581,406,78,8,228,NaN,NaN
4,pedropablofb,Pedro Pablo,,,2019-12-26,1210126217539706886,5,99,4,2,NaN,NaN


In [87]:
### rellenar los campos vacios  nan con 0
df_user_info_final["is_followers"].fillna(0,inplace=True)
df_user_info_final["is_followers"]=df_user_info_final["is_followers"].astype("int32")
df_user_info_final["is_following"].fillna(0,inplace=True)
df_user_info_final["is_following"]=df_user_info_final["is_following"].astype("int32")
# df_user_info_follow["is_followers"]=df_user_info_follow["is_followers"].map({True:1,False:0})
print(df_user_info_final.shape)
df_user_info_final.head()

(10852, 12)


,screen_name,username,location,blog,date_joined,user_id,num_tweets,following,followers,likes,is_followers,is_following
0,FelizDiaLima,#FelizDíaLima. 👑 👑 👑 ⭐️,,,2020-01-16,1218063509549993986,91,99,5,165,0,0
1,JuanCar99215124,Juan Carlos Miranda,,,2020-01-13,1216755584080347137,22,34,3,137,0,0
2,cocorocovillano,coqui villano diarrea,Jesus maria,,2020-01-10,1215664651813388289,200,249,49,342,0,0
3,CarasasJonathan,Jonathan Carasas C.,"Montevideo, Uruguay",elojocriticojcc.blogspot.com,2019-12-27,1210664626666520581,406,78,8,228,0,0
4,pedropablofb,Pedro Pablo,,,2019-12-26,1210126217539706886,5,99,4,2,0,0


In [0]:
df_user_info_final.to_pickle("../data/raw/df_user_info_final.pkl")